In [9]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
# Define some hyperparameters
batch_size = 32
epochs = 30
num_classes = 15
img_width, img_height = 224, 224

In [11]:
# Create an ImageDataGenerator to load the images and apply data augmentation

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
# Load the data from the directories
train_data = train_datagen.flow_from_directory(
    'split_data/train',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    'split_data/val',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

test_data = val_datagen.flow_from_directory(
    'split_data/test',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 5536 images belonging to 15 classes.
Found 686 images belonging to 15 classes.
Found 706 images belonging to 15 classes.


In [13]:
# Load the pre-trained MobileNet model with pre-trained weights from ImageNet
base_model = MobileNet(input_shape=(img_width, img_height, 3), include_top=False, weights='imagenet')

In [14]:
# Freeze the pre-trained layers to prevent their weights from being updated during training
for layer in base_model.layers:
    layer.trainable = False

In [15]:
# Add a new classification layer to the top of the pre-trained model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [16]:
# Compile the model with an appropriate optimizer and loss function
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
# Train the model on your training data
history = model.fit(train_data, epochs=epochs, validation_data=val_data)

Epoch 1/30
173/173 [==============================] - 181s 1s/step - loss: 1.8533 - accuracy: 0.3866 - val_loss: 1.6816 - val_accuracy: 0.4067
Epoch 2/30
173/173 [==============================] - 180s 1s/step - loss: 1.4794 - accuracy: 0.4910 - val_loss: 1.5713 - val_accuracy: 0.4723
Epoch 3/30
173/173 [==============================] - 164s 946ms/step - loss: 1.3707 - accuracy: 0.5325 - val_loss: 1.6265 - val_accuracy: 0.4315
Epoch 4/30
173/173 [==============================] - 156s 901ms/step - loss: 1.3101 - accuracy: 0.5461 - val_loss: 1.6260 - val_accuracy: 0.4242
Epoch 5/30
173/173 [==============================] - 177s 1s/step - loss: 1.2430 - accuracy: 0.5746 - val_loss: 1.4608 - val_accuracy: 0.4985
Epoch 6/30
173/173 [==============================] - 173s 1000ms/step - loss: 1.2533 - accuracy: 0.5750 - val_loss: 1.4825 - val_accuracy: 0.5117
Epoch 7/30
173/173 [==============================] - 170s 981ms/step - loss: 1.1756 - accuracy: 0.5990 - val_loss: 1.4195 - val_acc

In [18]:
# Evaluate the model on your test data
test_loss, test_acc = model.evaluate(test_data)
print('Test accuracy:', test_acc)

23/23 [==============================] - 15s 645ms/step - loss: 1.4336 - accuracy: 0.5425
Test accuracy: 0.5424929261207581


In [33]:
pip install scikit-multilearn

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Python39\python.exe -m pip install --upgrade pip' command.


In [35]:
from keras.utils import to_categorical
from sklearn.metrics import label_ranking_average_precision_score
from sklearn.metrics import top_k_accuracy_score

# Evaluate the model on the test set and make predictions
test_loss, test_acc = model.evaluate(test_data)
y_pred = model.predict(test_data)

# Compute LWLRAP score
y_true = to_categorical(test_data.labels, num_classes=15)
lwlrAP = label_ranking_average_precision_score(y_true, y_pred)
print('LWLRAP score:', lwlrAP)

# Compute top-5 categorical accuracy
top5_accuracy = top_k_accuracy_score(test_data.labels, y_pred, k=5)
print('Top-5 categorical accuracy:', top5_accuracy)


23/23 [==============================] - 12s 525ms/step
LWLRAP score: 0.29308048341334453
Top-5 categorical accuracy: 0.48441926345609065
